In [ ]:
import sys
sys.path.append('../code')
from init_mooc_nb import *
init_notebook()

import scipy
from matplotlib import cm
from matplotlib import gridspec        


sigma0 = np.array([[1, 0], [0, 1]])
sigmax = np.array([[0, 1], [1, 0]])
sigmay = np.array([[0, -1j], [1j, 0]])
sigmaz = np.array([[1, 0], [0, -1]])


class SimpleNamespace(object):
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

        
# Onsite and hoppings matrices used for building BHZ model
def onsite(site, par):
    A1, A2, B1, B2, D1, D2, C, M = par.A1, par.A2, par.B1, par.B2, par.D1, par.D2, par.C, par.M
    return (C + 2 * D1 + 4 * D2) * np.kron(sigma0, sigma0) + (M + 2 * B1 + 4 * B2) * np.kron(sigma0, sigmaz)


def hopx(site1, site2, par):
    A1, A2, B1, B2, D1, D2, C, M = par.A1, par.A2, par.B1, par.B2, par.D1, par.D2, par.C, par.M
    return - D2 * np.kron(sigma0, sigma0) - B2 * np.kron(sigma0, sigmaz) + A2 * 0.5j * np.kron(sigmax, sigmax)


def hopy(site1, site2, par):
    A1, A2, B1, B2, D1, D2, C, M = par.A1, par.A2, par.B1, par.B2, par.D1, par.D2, par.C, par.M
    return - D2 * np.kron(sigma0, sigma0) - B2 * np.kron(sigma0, sigmaz) + A2 * 0.5j * np.kron(sigmay, sigmax)


def hopz(site1, site2, par):
    A1, A2, B1, B2, D1, D2, C, M = par.A1, par.A2, par.B1, par.B2, par.D1, par.D2, par.C, par.M
    return - D1 * np.kron(sigma0, sigma0) - B1 * np.kron(sigma0, sigmaz) + A1 * 0.5j * np.kron(sigmaz, sigmax)


def bhz_infinite(h=5, finalized=True):
    """ Make infinite system for BHZ model """
    def shape(pos):
        (z,) = pos
        return (0 <= z < h)
    
    def transformed_onsite(site, par):
        kx,ky = par.kx, par.ky
        H0, H1, H2 = onsite(site, par), hopx(site, site, par), hopy(site, site, par)
        
        return H0  \
            + H1*np.exp(-1j*kx) + H1.transpose().conjugate()*np.exp(+1j*kx) \
            + H2*np.exp(-1j*ky) + H2.transpose().conjugate()*np.exp(+1j*ky)
            
    lat = kwant.lattice.chain()
    if h =='None':
        sym = kwant.TranslationalSymmetry((1,))
        sys = kwant.Builder(sym)
        sys[lat(0,)] = transformed_onsite
    else:
        sys = kwant.Builder()
        sys[lat.shape(shape, (0,))] = transformed_onsite
        
    sys[kwant.HoppingKind((1,), lat)] = hopz
    
    if finalized:
        return sys.finalized()
    else:
        return sys
    

def make_lead():
    lat = kwant.lattice.chain()
    sym = kwant.TranslationalSymmetry((-1,))
    sys = kwant.Builder(sym)
    sys[lat(0,)] = 1.5 * np.kron(sigma0, sigmaz)
    sys[kwant.HoppingKind((-1,), lat)] = -1.0 * np.kron(sigma0, sigmaz)
    return sys


def make_scatter_sys():
    lead_bhz = bhz_infinite('None', False)
    lead_n = make_lead()
    sys = bhz_infinite(1, False)
    sys.attach_lead(lead_bhz)
    sys.attach_lead(lead_n)
    fsys = sys.finalized()
    fsys.leads[1] = TRIInfiniteSystem(fsys.leads[1], trs)
    return fsys


def scattering_det_pfaff(fsys, ax):
    kx = [0, np.pi]
    kx_string = ['$k_x = 0$', '$k_x = \pi$']
    saved_kx = par.kx
    line_styles = [['r', 'ro'],['b', 'bo']]
    for i in range(len(kx)):
        pfaff =[]
        det = []
        par.kx = kx[i]
        par.ky = 0.0
        smat = kwant.smatrix(fsys, energy=0.0, args=[par]).submatrix(1, 1)
        #since we get relatively large numerical errors we project the matrix on the space of antisymmetric matrices
        smat = 0.5 * (smat - smat.T)
        pfaff.append(pf.pfaffian(smat))

        ks = np.linspace(0.0, np.pi, 100)
        for par.ky in ks:
            s = kwant.smatrix(fsys, energy=0.0, args=[par])
            smat = s.submatrix(1, 1)
            # Check that the system is gapped.
            assert smat.shape == s.data.shape
            det.append(np.linalg.det(smat))
        det = np.array(det)

        par.ky = np.pi
        smat = kwant.smatrix(fsys, energy=0.0, args=[par]).submatrix(1, 1)
        #since we get relatively large numerical errors we project the matrix on the space of antisymmetric matrices
        smat = 0.5 * (smat - smat.T)
        pfaff.append(pf.pfaffian(smat))
        
        ax.plot(ks[1:], np.angle(pfaff[0]) + 0.5 * np.cumsum(np.angle(det[1:]/det[:-1])), line_styles[i][0],label= kx_string[i])
        ax.plot([0, np.pi], np.angle(pfaff), line_styles[i][1])
    ax.set_ylim(-1.0, np.pi + 0.5)    
    ax.set_xlim(-0.2, np.pi+0.2)
    ax.set_xticks([0, np.pi])
    ax.set_xticklabels(['$0$', '$\pi$'])
    ax.set_yticks([ 0, np.pi])
    ax.set_yticklabels(['$0$', '$\pi$'])
    ax.set_xlabel('$k_y$')
    ax.set_ylabel('phase')
    ax.legend(bbox_to_anchor=(0.0, 1.02, 1.0, .102), loc=3,
       ncol=2, mode="expand", borderaxespad=0.)
    par.kx = saved_kx    

    
#Time reversal stuff
class TRIInfiniteSystem(kwant.builder.InfiniteSystem):
    def __init__(self, lead, trs):
        """A lead with time reversal invariant modes."""
        self.__dict__ = lead.__dict__
        self.trs = trs

    def modes(self, energy=0, args=()):
        prop_modes, stab_modes = \
            super(TRIInfiniteSystem, self).modes(energy=energy, args=args)
        n = stab_modes.nmodes
        stab_modes.vecs[:, n:(2*n)] = self.trs(stab_modes.vecs[:, :n])
        stab_modes.vecslmbdainv[:, n:(2*n)] = \
            self.trs(stab_modes.vecslmbdainv[:, :n])
        prop_modes.wave_functions[:, n:] = \
            self.trs(prop_modes.wave_functions[:, :n])
        return prop_modes, stab_modes
    
    
def trs(m):
    """Apply time reversal symmetry to a column vector or matrix m.

    The time reversal symmetry is given by the operator i * sigma_y * K, with K
    complex conjugation and sigma_y acting on the spin degree of freedom.

    Parameters:
    -----------
    m : NumPy array
        The vector or matrix to which TRS is applied.

    Returns:
    --------
    m_reversed : NumPy array
        The vector TRS * m as a NumPy array.

    Notes:
    ------
    Implementation inspired by kwant.rmt.
    """
    dim = m.shape[0]
    U = 1j * np.kron(np.identity(dim / 4), np.kron(sigmay, sigma0))
    
    return U.T.conj().dot(m.conj())
        

#Stuff for the bandstructure plot
def evaluate_on_grid(X, Y, func):
    """ X, Y should be in np.meshgrid form. It's enough for func to work on floats. """
    data = []
    for xx, yy in zip(X, Y):
        row = []
        for i,j in zip(xx, yy):
            row.append(func(i,j))
        data.append(row)
    data = np.array(data)
    data = list(np.transpose(data, (2, 0, 1)))
    return data


def dispersion_func(sys, par):
    def func(kx, ky):
        par.kx = kx
        par.ky = ky
        return diagonalize(sys, par)
    return func


def diagonalize(sys, par):
    mat = sys.hamiltonian_submatrix(args=[par])
    temp1 = len(mat)/2
    return scipy.linalg.eigvalsh(mat)[(temp1-2):(temp1+2)]  # Automatically sorted
    

def plot_2D(X,Y,Z, ax_in=None):
    if ax_in==None:
        fig = plt.figure(figsize=(7,5))
        ax = fig.add_subplot(111, projection='3d')
    else:
        ax = ax_in

    vmin = np.array(Z).min()
    vmax = np.array(Z).max()
    
    if len(np.shape(Z)) > 2:
        for z in Z:
            ax.plot_surface(X, Y, z, rstride=1, cstride=1, cmap=cm.RdBu_r, linewidth=0.1, vmin=vmin, vmax=vmax)
    else:
        ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.RdBu_r, linewidth=0.1, vmin=vmin, vmax=vmax)

    if ax_in==None:
        return fig, ax
    else:
        return ax

def plot_2d_dispersion(sys, ax):
    K = np.linspace(-np.pi, np.pi, 71)
    mesh = np.meshgrid(K, K)
    energies = evaluate_on_grid(*mesh, func=dispersion_func(sys, par))

    plot_2D(*mesh, Z=energies, ax_in=ax)

    ax.set_xlabel('$k_x$')
    ax.set_xlim3d(-np.pi, np.pi)
    ax.set_xticks([-np.pi, 0.0, np.pi])
    ax.set_xticklabels(['$-\pi$', '$0$', '$\pi$'])
    ax.set_ylabel('$k_y$')
    ax.set_yticks([-np.pi, 0.0, np.pi])
    ax.set_yticklabels(['$-\pi$', '$0$', '$\pi$'])
    ax.set_zlabel('$E$')
    ax.set_zticks([ -10, -5, 0, 5, 10])
    ax.set_zticklabels(['$-10$', '$-5$', '$0$', '$5$', '$10$'])
    ax.view_init(8,20)
    ax.set_zlim3d(-11, 11)   
    
    
def bhz_slab(L, W, H):
    lat = kwant.lattice.general(np.identity(3))
    sys = kwant.Builder()
    
    sym = kwant.TranslationalSymmetry((1, 0, 0))
    lead = kwant.Builder(sym)
    
    def shape_lead(pos):
        (x, y, z) = pos
        return (0 <= z < H) and (0 <= y < W)
    
    def shape(pos):
        (x, y, z) = pos
        return (0 <= z < H) and (0 <= y < W) and (0 <= x < L)
    
    def hopping_x(site1, site2, par):
        xt, yt, zt = site1.pos
        xs, ys, zs = site2.pos
        return hopx(site1,site2,par) * np.exp(-0.5j * par.Bz * (xt - xs) * (yt + ys))
    
    # scattering system
    sys[lat.shape(shape, (0,0,0))] = lambda site, par: onsite(site, par.scat) - par.mu * np.eye(4)
    sys[kwant.HoppingKind((1,0,0), lat)] = lambda site1, site2, par: hopping_x(site1, site2, par.scat)
    sys[kwant.HoppingKind((0,1,0), lat)] = lambda site1, site2, par: hopy(site1, site2, par.scat)
    sys[kwant.HoppingKind((0,0,1), lat)] = lambda site1, site2, par: hopz(site1, site2, par.scat)

    # leads
    lead[lat.shape(shape_lead, (0,0,0))] = lambda site, par: onsite(site, par.lead) - par.mu_lead * np.eye(4)
    lead[kwant.HoppingKind((1,0,0), lat)] = lambda site1, site2, par: hopping_x(site1, site2, par.lead)
    lead[kwant.HoppingKind((0,1,0), lat)] = lambda site1, site2, par: hopy(site1, site2, par.lead)
    lead[kwant.HoppingKind((0,0,1), lat)] = lambda site1, site2, par: hopz(site1, site2, par.lead)
    
    sys.attach_lead(lead)
    sys.attach_lead(lead.reversed())
    return sys.finalized()    

##Plan

* Making a 3D topological invariants out of 2D ones.
* BHZ model of a 3D topological insulator
* Dirac equation of the surface states and the fermion doubling
* Half-integer quantum Hall effect

#Table of Contents
* [Introduction](#Introduction)
* [Making  3D topological invariants out of 2D ones](#Making--3D-topological-invariants-out-of-2D-ones)
* [BHZ model of a 3D topological insulator](#BHZ-model-of-a-3D-topological-insulator)
* [Dirac surface states](#Dirac-surface-states)
* [Weak invariants](#Weak-invariants)
* [Quantum Hall conductance and the magneto-electric effect](#Quantum-Hall-conductance-and-the-magneto-electric-effect)
* [Conclusion: integers, half-integers, and two types of electromagnetic response](#Conclusion:-integers,-half-integers,-and-two-types-of-electromagnetic-response)


# Introduction

Joel Moore from the University of California, Berkeley will introduce this week's topic, by telling us how the idea of a two-dimensional topological insualtor was generalized to three dimensions.

In [ ]:
MoocVideo("NIhMjm7cyug", src_location="6.1-intro")

# Making  3D topological invariants out of 2D ones

Let us follow the direction explained by Joel Moore and construct a three-dimensional topological state from the two-dimensional topological state. This time, we'll do this by studying the system in momentum space rather than in real space as we did before. As with two dimensional systems, time-reversal invariant momenta (TRIMs) play an important role in three dimensions. 

For illustration, consider the three dimensional irreducible Brillouin Zone (i.e. $k_j\in [0,\pi]$) of a cubic system shown below. Fixing one of the three momenta $k_{x,y,z}$ to a TRIM, say $k_x=0$ without loss of generality, we can think of the Hamiltonian in the $(k_y,k_z)$ plane as a two dimensional Hamiltonian, which may either be topologically trivial ($\mathbb{Z}_2$-index $=0$) or non-trivial ($\mathbb{Z}_2$-index $=1$).


![](figures/3dbz.svg)

So for every side of the cube shown above we can compute a QSHE topological invariant, which gives us 6 numbers. However not all of them are independent. Specifically, there is a constraint $\mathcal{Q}(k_x=0)\,\mathcal{Q}(k_x=\pi) \equiv \mathcal{Q}(k_y=0)\,\mathcal{Q}(k_y=\pi) \equiv \mathcal{Q}(k_z=0)\,\mathcal{Q}(k_z=\pi)$.

This product is called *the strong topological invariant*, and accordingly the topological insulators where this invariant is non-trivial are strong topological insulators. The other three invariants can be chosen as $Q(k_x=\pi),\,Q(k_y=\pi),\,Q(k_z=\pi)$.

Very frequently the topological invariants of a compound are written as $(1;010)$, where the first number corresponds to the strong invariant, and the remaining three to the weak invariants along each axis. For example, the first predicted topological insulator, the alloy Bi$_x$Sb$_{1-x}$ is $(1;111)$, and the second generation topological insulators Bi$_2$Te$_3$ and Bi$_2$Se$_3$ are $(1;000)$.

Just by using the bulk-edge correspondence for $\mathcal{Q}$ we know that the strong topological invariant means that there is an odd number of helical states going in each direction on each facet of the topological insulator. We will see later why this is special, but before that let's construct a model for a 3D TI.

# BHZ model of a 3D topological insulator

Our goal in this unit is to derive an effective three-dimensional Hamiltonian $H(\mathbf{k})$ for a strong topological insulator.

We follow the same logic that led us to defining the three-dimensional topological invariant in the previous unit, building up on our knowledge of 2D topological insulators. Our first step is therefore to set $k_z=0$ and start from a two-dimensional Bloch Hamiltonian which describes a non-trivial 2D topological insulator. Of course, we choose a model we already know for this 2D Hamiltonian, the Bernevig-Hughes-Zhang (BHZ) model.

Let's recapitulate what we said about the BHZ model last week. It is a four band model, which has two electron bands (spin up and spin down) and two hole bands (spin up and down). It has inversion symmetry, with electron and hole bands having opposite parity. We will not need more bands for our 3D topological insulator model.

Copying the BHZ Hamiltonian of last week, at $k_z=0$ we have 

$$
H(k_z=0) = \epsilon(\mathbf{k})\cdot\mathbb{1} +
\begin{pmatrix}
M_0(\mathbf{k}) & A k_+ & 0 & 0 \\
A k_- & -M_0(\mathbf{k}) & 0 & 0 \\
0 & 0 & M_0(\mathbf{k}) & -Ak_- \\
0 & 0 & -Ak_+ & -M_0(\mathbf{k})
\end{pmatrix}\,,
$$

where $k_\pm = k_x \pm i k_y$. This Hamiltonian is written in a basis given by the states $\left|E\uparrow\right\rangle$, $\left|H\uparrow\right\rangle$, $\left|E\downarrow\right\rangle$, $\left|H\downarrow\right\rangle$, in this order. The block structure of the Hamiltonian reminds you that it is a doubled version of a Chern insulator, with two diagonal blocks for up and down spins. The particular form of $\epsilon(\mathbf{k})$ can be important to describe the band structure of a given material, but will not play a role in what follows. The effective mass is given by $M_0(\mathbf{k}) = M - B(k_x^2+k_y^2)$, and that changing the sign of $M$ you can describe the trivial and topological insulating phases

To get a strong topological insulator, we would like the two-dimensional $\mathbb{Z}_2$ invariant applied to the $(k_x, k_y)$ plane to take different values at $k_z=0$ and $k_z=\pi$. It is easy to achieve this by adding a $k_z$-dependent term to the effective mass, for instance in the following way

$$
M_0(\mathbf{k})\,\to\,M(\mathbf{k}) = M - B(k_x^2+k_y^2+k_z^2)\,.
$$

With this new mass term, for $k_z$ large enough and fixing $k_x$ and $k_y$ we can make the sign of the mass effectively change.

There is now a problem, however. As we follow the Hamiltonian at different values of $k_z$, we see that if we fix $k_z$, it still has time-reversal symmetry. Since it is topologically nontrivial at $k_z=0$ and becomes trivial at $k_z = \pi$, it should have a topological phase transition somewhere in between.

In other words, if we just add the $B k_z^2$ term, the Hamiltonian becomes gapless! Of course, we would like to have a gapped Hamiltonian in the whole 3D Brillouin zone instead.

How can we avoid the gap closing? We definitely have to couple the two spin blocks in $H(\mathbf{k})$, since otherwise each block is undergoing a Chern insulator transition. Since we couple the spins, to maintain time-reversal invariance of the system we need a coupling which is odd in momentum. The simplest thing is to choose the new term to be linear in $k_z$.

Hence we arrive at the following 3D Hamiltonian,

$$
H(\mathbf{k}) = \epsilon(\mathbf{k})\cdot\mathbb{1} +
\begin{pmatrix}
M(\mathbf{k}) & A k_+ & 0 & \tilde{A}k_z \\
A k_- & -M(\mathbf{k}) & \tilde{A}k_z & 0 \\
0 & \tilde{A}k_z & M(\mathbf{k}) & -Ak_- \\
\tilde{A}k_z & 0 & -Ak_+ & -M(\mathbf{k})
\end{pmatrix}\,,
$$

This Hamiltonian is known as the **3D BHZ model**. At finite $M$, it is gapped, and a phase transition between the trivial and strong topological insulator is achieved by changing the sign of $M$.  Just like its two-dimensional counterpart, it can be used as a prototype for a strong topological insulator, as well as a starting point to model real materials. 

The above derivation makes one important point evident: breaking spin conservation, which we used to avoid the undesirable gap closing at finite $k_z$, is a necessary ingredient to have a strong topological insulator.

# Dirac surface states

What is the dispersion of the surface state of the topological insulator?

We know that if we fix one momentum (say $k_x$) to zero, the Hamiltonian of the remaining system is that of a quantum spin Hall insulator.

For this system we know that the Hamiltonian of the edge states is just that of a pair of counter-propagating modes, so

$$H = v \sigma_y k_y.$$

The matrix $\sigma_y$ here acts on the degrees of freedom of these two surface modes, and doesn't correspond to a spin.

Since the time-reversal symmetry changes sign of $k_y$, it must also change sign of $\sigma_y$, so it has to be equal to $\mathcal{T} = i \sigma_y K$.

What if we set a finite $k_x$? Generically, the two modes will be coupled by an extra term in the Hamiltonian. This term should be proportional to $k_x$, and hence it must be also multiplied by a Pauli matrix, which we can just choose to be $\sigma_x$.

So if the surface of the topological insulator is isotropic, its Hamiltonian is merely

$$H=v \mathbf{\sigma} \cdot \mathbf{k}.$$

Let's just quickly take a look at it to get a more concrete understanding:

In [ ]:
par = SimpleNamespace(A1=1.0, A2=1.5, B1=1.0, B2=1.0, C=0.0, D1=0.0, D2=0.0, M=1.0,
                      kx=0.0, ky=0.0, Bz=0.0)

def plot_BHZ_dispersion(sys, M):

    par.M = M
    fig = plt.figure(figsize=(14, 6))
    ax1 = fig.add_subplot(121, projection='3d')
    plt.title('$M=%1.1f$' % par.M)
    plot_2d_dispersion(sys, ax=ax1)
    return fig

sys = bhz_infinite()
StaticInteract(lambda m: plot_BHZ_dispersion(sys, 0.5*m),
               m=RangeWidget(-2, 2))

What you see here is the dispersion of the two lowest energy bands of a thin slice of a 3D BHZ model.

As we expect, when system becomes topological ($M<0$), the surface states are formed, and the lowest energy state acquires a Dirac dispersion.

The distinguishing feature of the strong topological insulator is that it has an odd number of Dirac cones in total.
The reason why it is called "strong" is also the reason why having an odd number of Dirac cones is special.

To see what is unique let us compute the Chern number of the surface state if we add an infinitesimal magnetic field. We know that the number of Dirac cones is odd. Also from the Chern insulators we know that the *change* of the Chern number between $B = -\varepsilon$ and $B = +\varepsilon$ is just the number of the Dirac cones (we open a gap in each of them).

Since Chern number is odd under time reversal, we come to a paradoxical conclusion: if we break time reversal, on a surface of a topological insulator, we end up with a half-integer Hall conductance $\sigma_{xy} = e^2/h (n + 1/2)$.

This is of course not possible in any purely 2D system, since the Hall conductance must be an integer, and therefore the surface state of a strong topological insulator cannot be created without the topological bulk.

> The statement that it isn't impossible to have a tight-binding Hamiltonian with time-reversal symmetry and a single Dirac cone is known as the "fermion doubling theorem". There are several tricks that one can perform to work around this limitation in a numerical simulation, but we won't cover them in the course.

# Weak invariants

Now that we understand what is special about the strong invariant, let's deal with the weak invariants.

From their definition, we know that the weak invariants don't change the parity of the number of Dirac cones on any surface.
Furthermore, there is a very good reason why the weak invariants are called 'weak'. Imagine we keep the Hamiltonian the same, but instead we double the unit cell in each direction. That folds the Brillouin zone onto itself so that $k=\pi$ maps on $k=0$. This doubling of the unit cell doesn't impact the strong invariant, however all the weak invariants become 0.

As the final illustration of the relation between weak and strong invariants, let's see how the invariants change as a function of $M$ as we vary M on the scale comparable with the band width.

We use the same way to determine the topological invariant as for QSHE: we see if the phase of reflection matrix connects the Pfaffians of $r(k_y=0)$ and $r(k_y=\pi)$.

In [ ]:
par = SimpleNamespace(A1=1.0, A2=1.0, B1=1.0, B2=0.2, C=0.0, D1=0.1, D2=0.0, M=-1.5,
                      kx=0.0, ky=0.0, Bz=0.0)

sys = bhz_infinite(15)
fsys = make_scatter_sys()

def plot_dispersion_topology(sys, fsys, M):

    par.M = M
    
    fig = plt.figure(figsize=(12, 3.5))
    gs = gridspec.GridSpec(1, 2, width_ratios=[1.8, 1]) 
    ax1 = plt.subplot(gs[0], projection='3d')
    plt.title('$M=%1.2f$' %(par.M))
    ax2 = plt.subplot(gs[1])
    scattering_det_pfaff(fsys, ax2)
    plot_2d_dispersion(sys, ax=ax1)
    ax1.set_zlim3d(-3.1, 3.1)
    ax1.set_zticks([-3, 0, 3])
    ax1.set_zticklabels(['$-3$', '$0$', '$3$'])
    return fig

StaticInteract(lambda m: plot_dispersion_topology(sys, fsys, 0.5*m-2.75),
               m=RangeWidget(0, 7))

We see the values of the invariants change several times:

* Initially, when $M>0$, the system is trivial.
* Then, as $M$ is lowered, the topological invariants become $\mathcal{Q}(k_x=0) = 1$ and $\mathcal{Q}(k_x=\pi) = 0$, and there's a Dirac cone at $k=0$.
* When $M$ is lowered further, two new Dirac cones appear at $k = (0,\pi)$ and $k = (\pi, 0)$. This changes the invariants to $\mathcal{Q}(k_x=0) = 0$ and $\mathcal{Q}(k_x=\pi) = 1$.
* Finally one more Dirac cone appears at $k = (\pi, \pi)$, accompanied with both invariants becoming trivial.

In [ ]:
question = ("Suppose you have a $(0;100)$ weak topological insulator. Which one of the following statements is correct?")

answers = ["There is an even number of Dirac cones for both $k_x=0$ and $k_x=\pi$.",
           "There is an even number of Dirac cones for $k_x=0$ and an odd one for $k_x=\pi$.",
           "There is an odd number of Dirac cones for $k_x=0$ and an even one for $k_x=\pi$.",
           "There is an odd number of Dirac cones for both $k_x=0$ and $k_x=\pi$."]

explanation = ("We know that the strong invariant $Q(k_x=0)Q(k_x=\pi)=0$, so there must be an even number of Dirac cones in total. "
               "The number at $k_x=\pi$ is odd because $Q(k_x=\pi)=1$, so also the number at $k_x=0$ must be odd.")

MoocMultipleChoiceAssessment(question=question, answers=answers, correct_answer=3, explanation=explanation)

# Quantum Hall conductance and the magneto-electric effect

So. What happens if we try to measure the Hall conductance of a 3D topological insulator? Will we actually obtain a half-integer? If you followed the course closely, you will know that the answer is negative, and that the Hall conductance is always quantized to an integer. So what is going on if we try to measure the Hall conductance?

To answer this question we need to remember that we cannot just consider one surface on its own. Each surface is connected to the sides and other surfaces. So let's consider a sample of a 3D TI in magnetic field:

![](figures/3d_ti_slab.svg)

The top surface has quantum Hall conductance of $(n + 1/2)e^2/h$, but it's not the only part of the material. The bottom surface is also subject to magnetic field and also contributes a half-integer Hall conductance.

So in total we get $\sigma_{xy} = (2n + 1) e^2/h$: an integer, which resolves the apparent paradox. While this statement sounds relatively simple, there are entire papers dedicated to the question if it is possible to measure a half-integer Hall conductance (the answer is 'not really').

Finally, let's look at the dispersion of the Landau levels and the edge states:

In [ ]:
Bz = 0.125
mu_lead = 0.7
mu_max = .7

L, W, H = 10, 20, 10
sys = bhz_slab(L, W, H)

momenta = np.linspace(-1.5, 3.5, 60)
par_lead = SimpleNamespace(A1=1.0, A2=1.0, B1=1.0, B2=1.0, C=0.0, D1=0., D2=0., M=-1.0, Bz=Bz)
par = SimpleNamespace(lead=par_lead, scat=par_lead, mu=0.0, mu_lead=0.0)
fig, ax = plt.subplots(figsize=((8,6)))
ax.set_color_cycle(['k'])
    
kwant.plotter.bands(sys.leads[0], args=[par], momenta=momenta, show=False, ax=ax);

ax.set_ylim(-0.8,0.8);
ax.set_xlabel('$k$');
ax.set_ylabel('$E$');

vals = np.arange(-2, 4);
ax.set_xticks(vals);
ax.set_xticklabels(["${0}$".format(i) for i in vals]);

vals = np.arange(-0.8, 1.2, 0.4);
ax.set_yticks(vals);
ax.set_yticklabels(["${0}$".format(i) for i in vals]);

You see pairs of Landau levels. In each pair one level comes from the top surface, one from the bottom surface. Magnetic field is parallel to the side surfaces, so that there is no gap there. The edge states propagate freely along the side surfaces and are reflected by the magnetic field as they try to enter either top or the bottom.

In [ ]:
question = ("Suppose that you take the 3D TI slab above, and connect the left and right surfaces, making it into "
            "a very thick Corbino disk. "
            "You then apply to it a strong perpendicular field in the same direction as in the figure, perpendicular to the top "
            "and bottom surfaces. "
            "What happens if you throw an additional flux quantum through the inner hole of the disk?")

answers = ["A half-integer amount of electron charges is transferred from the inner to the outer surface of the disk.",
           "An integer of electron charges is transferred from the inner to the outer surface of the disk.",
           "An integer number of charges is transferred from the top to the bottom surface of the disk.",
           "The bulk gap closes."]

explanation = ("The top and bottom surfaces combined form an integer quantum Hall state. "
               "Hence the whole system acts like a Laughlin pump, exactly like in the purely 2D case.")

MoocMultipleChoiceAssessment(question=question, answers=answers, correct_answer=1, explanation=explanation)

# Conclusion: integers, half-integers, and two types of electromagnetic response

Before we move on to the next lecture, Joel Moore will tell us more about the origins of the peculiar electromagnetic response of topological insulators, and a fascinating connection to high energy physics.

In [ ]:
MoocVideo("s7H6oLighOM", src_location="6.1-summary")

In [ ]:
MoocDiscussion("Questions", "3DTI invariants")